In [ ]:
!pip install pandas pyarrow

In [1]:
import shutil
import pandas as pd
import numpy as np
import os 

# Leitura dos arquivos


In [2]:
#trazendo os dados da bronze
origem = "C:\\Users\\jhoe.hashimoto\\Desktop\\v2dados_teste\\bronze\\parquet\\"

# Lista todos os arquivos no diretório
arquivos = os.listdir(origem)

# Filtra apenas os arquivos .xlsx ou excel
arquivos_parquet = [arquivo for arquivo in arquivos if arquivo.endswith('.parquet')]

# Lista para armazenar os DataFrames
dataframes = []

# Lê cada arquivo .xlsx e armazena na lista
for arquivo in arquivos_parquet:
    caminho_completo = os.path.join(origem, arquivo)
    df = pd.read_parquet(caminho_completo)
    dataframes.append(df)

customer1, customer2, sales1, sales2= dataframes

# Junção tabelas Versão #1

In [4]:
#Junção das tabelas
df_combined1 = pd.merge(customer1, sales1, on='id_cliente')
df_combined1.head(n=5)

,id_cliente,nome,sobrenome,email,telefone,endereço,numero,order_id,product_name,quantity,unit_price,order_date
0,1,João,Silva,joao.silva@email.com,(11) 1234-5678,Rua das Flores,123,56,Produto C,2,20.0,2024-04-25
1,2,Maria,Santos,maria.santos@email.com,(11) 2345-6789,Avenida Principal,456,51,Produto B,3,30.0,2024-04-20
2,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,35,Produto B,4,30.0,2024-04-04
3,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,88,Produto C,1,20.0,2024-05-27
4,4,Ana,Costa,ana.costa@email.com,(11) 4567-8901,Rua dos Passarinhos,987,37,Produto D,3,40.0,2024-04-06


In [ ]:
df_combined1.info()

# Junção tabelas Versão #2

In [6]:
df_combinedcarga2 = pd.merge(customer2,sales2, on='id_cliente')
df_combinedcarga2.head()

,id_cliente,nome,sobrenome,email,telefone,endereço,numero,Data_Cadastro,order_id,product_name,quantity,unit_price,order_date
0,1,João,Silva,joao.silva@email.com,(11) 1234-5678,Rua das Flores,123,2020-02-03,56,Produto C,2,20.0,2024-04-25
1,2,Maria,Santos,maria.santos@email.com,(11) 2345-6789,Avenida Principal,456,2023-05-22,51,Produto B,3,30.0,2024-04-20
2,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,2020-01-18,35,Produto B,4,30.0,2024-04-04
3,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,2020-01-18,88,Produto C,1,20.0,2024-05-27
4,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,2020-01-18,110,Produto C,1,20.0,2024-05-27


In [ ]:
df_combinedcarga2.info()

***UPDATE DOS DADOS ATRAVÉS DO PANDAS***   

###### Necessidade validação*!

In [9]:
# Definindo a coluna 'id' como índice
df_combined1.set_index('order_id', inplace=True)
df_combinedcarga2.set_index('order_id', inplace=True)

# Passo 1: Atualizar df1 com os valores de df2
df_combined1.update(df_combinedcarga2)

# Passo 2: Identificar e remover as linhas de df1 que não estão em df2
# Aqui, estamos removendo as linhas que não têm o índice presente em df2
indices_to_keep = df_combinedcarga2.index
df_combined1 = df_combined1[df_combined1.index.isin(indices_to_keep)]

# Passo 3: Adicionar novas linhas de df2 que não estão em df1
# Aqui, usamos a coluna `combine_first` para adicionar as novas linhas
df_upsert = df_combined1.combine_first(df_combinedcarga2)

# Resetando o índice para a coluna 'order_id'
df_upsert.reset_index(inplace=True)

In [17]:
df_combinedcarga2['Data_Cadastro']

0     2020-02-03
1     2023-05-22
2     2020-01-18
3     2020-01-18
4     2020-01-18
         ...    
161   2021-03-18
162   2021-03-18
163   2021-03-18
164   2023-12-25
165   2022-03-30
Name: Data_Cadastro, Length: 166, dtype: datetime64[ns]

In [15]:
df_combined1.reset_index(inplace=True)
df_combinedcarga2.reset_index(inplace=True)

 *Normalização de Colunas*

In [19]:
#RENOMEIO E PADRONIZAÇÃO DAS COLUNAS ENDERECO
for df in dataframes:
    if 'endereço' in df.columns:
        df.rename(columns={'endereço': 'endereco'}, inplace=True)

df_combined1.rename(columns={'endereço': 'endereco'}, inplace=True)
df_combinedcarga2.rename(columns={'endereço': 'endereco'}, inplace=True)
df_upsert.rename(columns={'endereço': 'endereco'}, inplace=True)

# CONEXÃO COM O MYSQL

In [ ]:
!pip install mysql-connector-python

In [20]:
from mysql.connector import connect
import mysql.connector
from mysql.connector import errorcode

In [21]:
#dados servidor
config = {
    'user': 'root',
    'password': 'jhoehashimoto',
    'host': 'localhost',
    'database': 'V2silver_sales_customer'
}

# INÍCIO DA MODELAGEM 

* CRIANDO AS TABELAS

In [14]:
#tabela join#1
try:
    # Criando a conexão
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()

        # Criando o schema
        join_sales_customer_silver = """
                        CREATE TABLE IF NOT EXISTS v1join_sales_customer_silver (
                            id_cliente int NOT NULL,
                            nome varchar(50),
                            sobrenome varchar(50),
                            email varchar(50),
                            telefone varchar(20),
                            endereco varchar(50),
                            numero int,
                            order_id int,
                            product_name varchar(50),
                            quantity int,
                            unit_price float,
                            order_date date,
                            PRIMARY KEY (order_id)
                        ) 
        """
        
        cursor.execute(join_sales_customer_silver)
        print('Tabela criada com sucesso')
        
        # Fechando o cursor e a conexão
        cursor.close()
        connection.close()
    else:
        print('Não foi possível conectar ao banco de dados')
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("A tabela já existe.")
    else:
        print(f"Erro: {err.msg}")
finally:
    # Garantir que a conexão seja fechada se aberta
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')

Tabela criada com sucesso


* CRIANDO TABELAS VERSÃO

In [14]:
#tabela join####2
try:
    # Criando a conexão
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()

        # Criando o schema
        v2join_sales_customer_silver = """
                        CREATE TABLE v2join_sales_customer_silver (
                            id_cliente int NOT NULL,
                            nome varchar(50),
                            sobrenome varchar(50),
                            email varchar(50),
                            telefone varchar(20),
                            endereco varchar(50),
                            numero int,
                            order_id int,
                            product_name varchar(50),
                            quantity int,
                            unit_price float,
                            order_date date,
                            Data_Cadastro date,
                            PRIMARY KEY (order_id)
                        ) 
        """
        
        cursor.execute(v2join_sales_customer_silver)
        print('Tabela criada com sucesso')
        
        # Fechando o cursor e a conexão
        cursor.close()
        connection.close()
    else:
        print('Não foi possível conectar ao banco de dados')
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("A tabela já existe.")
    else:
        print(f"Erro: {err.msg}")
finally:
    # Garantir que a conexão seja fechada se aberta
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')

Tabela criada com sucesso


TRADUZINDO TIPAGEM PY PARA O MYSQL

In [22]:
import numpy as np
def pandas_to_mysql_type(value):
    if isinstance(value, np.int64):
        return int(value)
    elif isinstance(value, np.float64):
        return float(value)
    elif isinstance(value, pd.Timestamp):
        return value.to_pydatetime().strftime('%Y-%m-%d %H:%M:%S')
    elif isinstance(value, pd.Series):
        return value.values.tolist()
    else:
        return value

# Populando as tabelas

In [23]:
customer1, customer2, sales1, sales2= dataframes

In [29]:
df_combined1.head(n=5)

,index,order_id,id_cliente,nome,sobrenome,email,telefone,endereco,numero,product_name,quantity,unit_price,order_date
0,0,56,1,João,Silva,joao.silva@email.com,(11) 1234-5678,Rua das Flores,123,Produto C,2,20.0,2024-04-25
1,1,51,2,Maria,Santos,maria.santos@email.com,(11) 2345-6789,Avenida Principal,456,Produto B,3,30.0,2024-04-20
2,2,35,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,Produto B,4,30.0,2024-04-04
3,3,88,3,Carlos,Rocha,carlos.rocha@email.com,(11) 3456-7890,Travessa das Árvores,789,Produto C,1,20.0,2024-05-27
4,4,37,4,Ana,Costa,ana.costa@email.com,(11) 4567-8901,Rua dos Passarinhos,987,Produto D,3,40.0,2024-04-06


In [ ]:
#insert tabela v1join_sales_customer_silver
try:
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()
        
        # Inserir os dados da tabela df_combined1 na tabela v1join_sales_customer_silver
        for i in df_combined1.index:
            query = """
            INSERT INTO v1join_sales_customer_silver (id_cliente, nome, sobrenome, email, telefone, numero, endereco, order_id, product_name, quantity, unit_price, order_date)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            data = (
                pandas_to_mysql_type(df_combined1['id_cliente'][i]),
                df_combined1['nome'][i],
                df_combined1['sobrenome'][i],
                df_combined1['email'][i],
                df_combined1['telefone'][i],
                pandas_to_mysql_type(df_combined1['numero'][i]),
                df_combined1['endereco'][i],
                pandas_to_mysql_type(df_combined1['order_id'][i]),
                df_combined1['product_name'][i],
                pandas_to_mysql_type(df_combined1['quantity'][i]),
                pandas_to_mysql_type(df_combined1['unit_price'][i]),
                pandas_to_mysql_type(df_combined1['order_date'][i]),
            )
            
            cursor.execute(query, data)
            print(f'Dados inseridos com sucesso')
        
        # Commit das alterações e fechamento do cursor
        connection.commit()
        cursor.close()
        print('Commit realizado com sucesso')
    
except mysql.connector.Error as err:
    print(f"Erro MySQL: {err}")
    if connection.is_connected():
        connection.rollback()
finally:
    # Fechamento da conexão
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')

In [ ]:
#insert tabela v2join_sales_customer_silver
try:
    connection = mysql.connector.connect(**config)
    
    if connection.is_connected():
        cursor = connection.cursor()
        
        # Inserir os dados da tabela df_combinedcarga2 na tabela v2join_sales_customer_silver
        for i in df_combinedcarga2.index:
            query = """
            INSERT INTO v2join_sales_customer_silver (id_cliente, nome, sobrenome, email, telefone, numero, endereco, order_id, product_name, quantity, unit_price, order_date, Data_Cadastro)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            data = (
                pandas_to_mysql_type(df_combinedcarga2['id_cliente'][i]),
                df_combinedcarga2['nome'][i],
                df_combinedcarga2['sobrenome'][i],
                df_combinedcarga2['email'][i],
                df_combinedcarga2['telefone'][i],
                pandas_to_mysql_type(df_combinedcarga2['numero'][i]),
                df_combinedcarga2['endereco'][i],
                pandas_to_mysql_type(df_combinedcarga2['order_id'][i]),
                df_combinedcarga2['product_name'][i],
                pandas_to_mysql_type(df_combinedcarga2['quantity'][i]),
                pandas_to_mysql_type(df_combinedcarga2['unit_price'][i]),
                pandas_to_mysql_type(df_combinedcarga2['order_date'][i]),
                pandas_to_mysql_type(df_combinedcarga2['Data_Cadastro'][i]),
            )
            
            cursor.execute(query, data)
            print(f'Dados inseridos com sucesso')
        
        # Commit das alterações e fechamento do cursor
        connection.commit()
        cursor.close()
        print('Commit realizado com sucesso')
    
except mysql.connector.Error as err:
    print(f"Erro MySQL: {err}")
    if connection.is_connected():
        connection.rollback()
finally:
    # Fechamento da conexão
    if connection.is_connected():
        connection.close()
        print('Conexão ao MySQL fechada')